# AV Calculator Python

Things I don't understand:
- why is tier here separated into 1 or 2? This is different than the metal plans. So then why is the 1 or 2 used to calculate tier specific inputs?
- csr levels (cost sharing reduction)
- in avc_1_main, why is there a for loop causing it to run twice?

Methods:
- avc_1_main
- avc_2-1_combined
- avc_2-2_sep_deduct_comb_moop
- avc_2-3_sep_deduct_sep_moop
- avc_2_globals
- avc_3_on_click
- avc_4_validate
- avc_5_formatting
- avc_6_create_image
- avc_7_initialize
- avc_8_functions
- avc_9_plan_data
- avc_10_process_plan_data
- avc_11_set_cont_tables
- avc_12_hsa
- avc_13_reset_vars
- avc_14_calc_av
- avc_15_process_av

In [1]:
class AV_Calculator:
    def __init__(self): 
        self.avc_2_globals()
    
    def avc_1_main(self):
        av_1 = 0
        av_2 = 0
        self.avc_9_plan_data()
        self.avc_4_validate()    
        # I am not sure why this runs twice, but it is this way in the Excel macros.
        for x in range(1,2):
            if self.gCB_MULTITIER_PLAN == 1: # if it's a multitier plan, calculate the weighted average of the AV's
                self.get_cost_sharing_parameters(1)
                self.avc_4_validate(1)
                av_1 = self.avc_14_calc_av(1)
                self.get_cost_sharing_parameters(2)
                self.avc_4_validate(2)
                av_2 = self.avc_14_calc_av(2)
                self.gAV = self.gTIER1_UTIL * av_1 + self.gTIER2_UTIL * av_2   
            else: # otherwise, just calculate the first AV
                self.get_cost_sharing_parameters(1)
                self.validate_tier_specific_inputs(1)
                self.gAV = self.avc_14_calc_av(tier_1)          
            if self.gCB_HSA_IND == 1: # adjust for the hsa index (?)
                self.gAV = self.avc_12_hsa()
            self.gAV = min(1, self.gAV)  # cap AV at 1, make sure that AV is valid
            metal = self.avc_15_process_av(self.gAV)
            if self.gACTUAL_MTIER_NUM == self.gDESIRED_MTIER_NUM or self.gACTUAL_MTIER_NUM <= 0 or self.gCB_CSR_IND == 1:
                break
            else:
                print("desired tier does not match with calculated metal tier")
                
    def avc_21_combined(self):
        PRIOR_COINS = 0
        COINS = 1
        ACTL_COINS_RATE_ACHVD = -1
        AMT_STD = 0
        ded_maxd = 0
        dedrow = 0
       
        EFF_COINS_DENOM = 0
        EFF_COINS_DENOM = gMaxdCol.Rng(1).Cells(gBottomRow, 1)
    
        #The following code iteratively calculates the amount of spending required to meet the deductible.  Cost sharing below
        # the deductible means that less than 100% of service costs is being born by the beneficiary; hence a higher amount of
        # absolute spending is required to satisfy the deductible.
        it = 0
        while it < self.gMAX_CONVERGENCE_ITER and Abs(PRIOR_COINS - ACTL_COINS_RATE_ACHVD) > self.gTOLERANCE:    
            self.gMOOP_ADJMT = 0
            self.gPLAN_PAY = 0
            self.gBENEPAY = 0
            self.gTOT_PAY = 0
            self.gEFF_COINS_NUM = 0
            
            if COINS == 0:
                self.gADJ_DEDUCT = self.gADJ_DEDUCT
            else:
                self.gADJ_DEDUCT = self.gDEDUCT / COINS
        
            # determine row of deductible in continuance table
            dedrow = getContTableRow(gUpToCol.Rng(1), gADJ_DEDUCT)

            # MEDICAL SERVICES PROCESSING
            # Emergency Room Services
            ProcessServiceCostShare(gER, dedrow, gER_COPAY, gER_COINS, it)
            # Inpatient
            ProcessServiceCostShare(gIP, dedrow, gIP_COPAY, gIP_COINS, it)
            # Primary Care Visit
            if gCB_OFV_LIMIT_IND = 0 and gCB_OFV_COPAY_LMT_IND = 0:
                ProcessServiceCostShare(gPC, dedrow, gPC_COPAY, gPC_COINS, it)
            else: # PC Additional Plan Options
                if gCB_OFV_LIMIT_IND = 1:
                    #'first N visits are free - essentially have a copay of $0
                    ProcessTruncServiceVal(gPC, gPCN, dedrow, 0, 1, it)
                    # cost sharing after N visits
                    ProcessServiceCostShare(gPCN, dedrow, gPC_COPAY, gPC_COINS, it)
                elif: gCB_OFV_COPAY_LMT_IND = 1:
                    # first N visits paid with copay
                    ProcessTruncServiceVal(gPC, gPCN, dedrow, gPC_COPAY, 1, it)
                    # cost sharing after N visits below the deductible - Assume no cost-sharing, so set copay to 0.
                    ProcessServiceCostShare(gPCN, dedrow, 0, gPC_COINS, it)
            # Specialist Visit
            ProcessServiceCostShare(gSP, dedrow, gSP_COPAY, gSP_COINS, it)
            # Mental/Behavioral Health and Substance Abuse Disorder Oupatient Services
                # Outpatient Facility Fee Component
                # -- Use OP Facility cost sharing unless primary service has special cost sharing.
                if gOPFAC_PSY_OVRD = 1:
                    ProcessServiceCostShare(gOPFAC_PSY, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
                else:
                    ProcessServiceCostShare(gOPFAC_PSY, dedrow, gPSY_COPAY, gPSY_COINS, it)
                if gOPPROF_PSY_OVRD = 1:
                    ProcessServiceCostShare(gOPPROF_PSY, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
                else:
                    ProcessServiceCostShare(gOPPROF_PSY, dedrow, gPSY_COPAY, gPSY_COINS, it)
            # Imaging
                # Outpatient Facility Fee Component
                # -- Use OP Facility cost sharing unless primary service has special cost sharing.
                if gOPFAC_IMG_OVRD = 1:
                    ProcessServiceCostShare(gOPFAC_IMG, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
                else:
                    ProcessServiceCostShare(gOPFAC_IMG, dedrow, gIMG_COPAY, gIMG_COINS, it)
                # Outpatient Services Component
                # -- Use OP Professional cost sharing unless primary service has special cost sharing.
                if gOPPROF_IMG_OVRD = 1:
                    ProcessServiceCostShare(gOPPROF_IMG, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
                else:
                    ProcessServiceCostShare(gOPPROF_IMG, dedrow, gIMG_COPAY, gIMG_COINS, it)
            # Rehabilitative Speech Therapy
                # Outpatient Facility Fee Component
                # -- Use OP Facility cost sharing unless primary service has special cost sharing.
                if gOPFAC_ST_OVRD = 1:
                     ProcessServiceCostShare(gOPFAC_ST, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
                else:
                    ProcessServiceCostShare(gOPFAC_ST, dedrow, gST_COPAY, gST_COINS, it)
                # Outpatient Services Component
                # -- Use OP Professional cost sharing unless primary service has special cost sharing.
                if gOPPROF_ST_OVRD = 1:
                    ProcessServiceCostShare(gOPPROF_ST, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
                else:
                    ProcessServiceCostShare(gOPPROF_ST, dedrow, gST_COPAY, gST_COINS, it)
            # Rehabilitative Occupational and Rehabilitative Physical Therapy
                # Outpatient Facility Fee Component
                # -- Use OP Facility cost sharing unless primary service has special cost sharing.
                if gOPFAC_OT_OVRD = 1:
                    ProcessServiceCostShare(gOPFAC_OT, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
                else:
                    ProcessServiceCostShare(gOPFAC_OT, dedrow, gOT_COPAY, gOT_COINS, it)
                # 'Outpatient Services Component
                # -- Use OP Professional cost sharing unless primary service has special cost sharing.
                if gOPPROF_OT_OVRD = 1:
                    ProcessServiceCostShare(gOPPROF_OT, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
                else:
                    ProcessServiceCostShare(gOPPROF_OT, dedrow, gOT_COPAY, gOT_COINS, it)
            # Preventive Care
                ProcessServiceCostShare(gPREV, dedrow, gPREV_COPAY, gPREV_COINS, it)
            # Laboratory Outpatient and Professional Services
                # Outpatient Facility Fee Component
                # -- Use OP Facility cost sharing unless primary service has special cost sharing.
                if gOPFAC_LAB_OVRD = 1:
                    ProcessServiceCostShare(gOPFAC_LAB, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
                else:
                    ProcessServiceCostShare(gOPFAC_LAB, dedrow, gLAB_COPAY, gLAB_COINS, it)
                # Outpatient Services Component
                # -- Use OP Professional cost sharing unless primary service has special cost sharing.
                if gOPPROF_LAB_OVRD = 1:
                    ProcessServiceCostShare(gOPPROF_LAB, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
                else:
                    ProcessServiceCostShare(gOPPROF_LAB, dedrow, gLAB_COPAY, gLAB_COINS, it)
            # 'X-rays and Diagnostic Imaging
                # If X-Ray has its own cost sharing paremeters, then these take precedence over
                # X-Ray component cost sharing paremeters.  If X-Ray does not have its own cost
                # sharing parameters, break X-Ray into component parts and apply component cost
                # sharing parameters.
                if gXRAY_OVRD:
                    ProcessServiceCostShare(gXRAY, dedrow, gXRAY_COPAY, gXRAY_COINS, it)
                else:
                    # 'X-rays and Diagnostic Imaging - Primary Care Visit Component
                    if gCB_OFV_LIMIT_IND = 1:
                        # Special case when "Begin PC Cost-sharing After N Visits" additional benefit design option is enabled
                        ProcessCostShare_PCXRAY_NVisits(vTABLE:=gXRAY_PC, vDEDROW:=dedrow, vCOPAY_NVISITS:=0, vCOPAY_COINS_RNG:=gPC_COPAY, vCOINS:=gPC_COINS, vITER:=it)    
                    elif gCB_OFV_COPAY_LMT_IND = 1:
                        # Special case when "Begin PC Deduct/Coins After N Visits" additional benefit design option is enabled
                        ProcessCostShare_PCXRAY_NVisits(vTABLE:=gXRAY_PC, vDEDROW:=dedrow, vCOPAY_NVISITS:=gPC_COPAY, vCOPAY_COINS_RNG:=gPC_COPAY, vCOINS:=gPC_COINS, vITER:=it)
                    else:
                        ProcessServiceCostShare(gXRAY_PC, dedrow, gPC_COPAY, gPC_COINS, it)
                    # X-rays and Diagnostic Imaging - Specialist Visit Component
                        ProcessServiceCostShare(gXRAY_SP, dedrow, gSP_COPAY, gSP_COINS, it)
                    # X-Rays and Diagnostic Imaging - Unclassified
                        ProcessServiceCostShare(gXRAY_UNC, dedrow, gXRAY_COPAY, gXRAY_COINS, it)
            # 'Skilled Nursing Facility
                ProcessServiceCostShare(gSNF, dedrow, gSNF_COPAY, gSNF_COINS, it)
            # Unclassified OPFAC/OPPROF
            #Outpatient Facility Fee Component
                ProcessServiceCostShare(gOPFAC_UNC, dedrow, gOPFAC_COPAY, gOPFAC_COINS, it)
            # Outpatient Services Component
                ProcessServiceCostShare(gOPPROF_SRGRY, dedrow, gOPPROF_COPAY, gOPPROF_COINS, it)
            # 'DRUG PROCESSING
                # Generics
                ProcessServiceCostShareRX(gRXGEN, dedrow, gRXGEN_COPAY, gRXGEN_COINS, it)
                # Preferred Brand Drugs
                ProcessServiceCostShareRX(gRXFORM, dedrow, gRXFORM_COPAY, gRXFORM_COINS, it)
                # Non-Preferred Brand Drugs
                ProcessServiceCostShareRX(gRXNONFORM, dedrow, gRXNONFORM_COPAY, gRXNONFORM_COINS, it)
                # Specialty Drugs
                ProcessServiceCostShareRX(gRXSPCLTY, dedrow, gRXSPCLTY_COPAY, gRXSPCLTY_COINS, it)

            # Recalculate deductible values
            AMT_STD = gTOT_PAY
            if AMT_STD = 0:
                ACTL_COINS_RATE_ACHVD = 1
            else:
                ACTL_COINS_RATE_ACHVD = gBENEPAY / AMT_STD

            PRIOR_COINS = COINS
            #Update for next run of loop
            #This update step initially weights the actual coinsurance rate achieved highly, but as
            #the number of iterations climbs and the algorithm fails to converge, the update step
            #puts additional weight on the average of the old and new coinsurance rates.
            COINS = ACTL_COINS_RATE_ACHVD * Exp(-it / gTUNING_PRMTR) + _
                (PRIOR_COINS + ACTL_COINS_RATE_ACHVD) / 2 * (1 - Exp(-it / gTUNING_PRMTR))
            it = it + 1

        # Set plan paid amount equal to portion of TOTAL spending below the deductible that the plans pays for.
        ded_maxd = computeRowVal(gMaxdCol.Rng(1), dedrow)
        if gTOT_PAY = 0: 
            gPLAN_PAY = 0
        else:
            gPLAN_PAY = ded_maxd * gPLAN_PAY / gTOT_PAY

        # Set variables to use in HSA Contribution modification, if needed.
        if gTOT_PAY = 0:
            gBENECOINS_DEDUCT = 1
        else: gBENECOINS_DEDUCT = gBENEPAY / gTOT_PAY
        
        # True MOOP Calculation - allowed spending required for enrollee to reach his or her MOOP
        gEFF_COINS = Application.Min(gEFF_COINS_NUM / EFF_COINS_DENOM, 1)
        if gEFF_COINS < 0 or gEFF_COINS > 1:
            raise Exception("ERROR - Effective Coinsurance OOB")
    
        gADJ_MOOP = gMOOP - gMOOP_ADJMT
        
        if gADJ_MOOP < gDEDUCT:
            gADJ_MOOP = gDEDUCT
        
        if gEFF_COINS = 1:
            gTROOP = gADJ_DEDUCT
        else:
            # Note that the coinsurance range is always calculated based on the inputted deductible.  This is because this
            # is the direct amount of spending required to satisfy the deductible range and hence reflects the amount of
            # MOOP spending that has been met so far.
            gTROOP = gADJ_DEDUCT + (gADJ_MOOP - gDEDUCT) / (1 - gEFF_COINS)
        
        gDEDROW = dedrow


    
    def avc_22_sep_deduct_comb_moop:
        pass
    
    def avc_23_sep_deduct_sep_moop:
        pass
    
    # define all of the global variables
    def avc_2_globals:
        self.avc_13_reset_vars()
    
    def avc_3_on_click:
        pass
    
    def avc_4_validate:
        pass
    
    def avc_5_formatting:
        pass
    
    def avc_6_create_image:
        pass
    
    def avc_7_initialize:
        pass
    
    def avc_8_functions:
        pass
    
    def avc_9_plan_data:
        pass
    
    def avc_10_process_plan_data:
        pass
    
    def avc_11_set_cont_tables:
        pass
    
    #hsa stands for health savings account i think
    def avc_12_hsa(self):
        hsa_row = 0
        if benecoins_deduct != 0:
            row = hsa_contrbtn / benecoins_deduct
        
        # get correct hsa row from table 
        #HSA_VALUE = computeRowVal(gMaxdCol.Rng(1), HSA_ROW) * gBENECOINS_DEDUCT
        #AV_DENOM = gMaxdCol.Rng(1).Cells(gBottomRow, 1)
        
        # then do math
        #BENEPAY = (1 - vAV) * AV_DENOM
        #HSA_ADJ = Application.Min(BENEPAY, HSA_VALUE)   
        #HSA_MOD_AV = (vAV * AV_DENOM + HSA_ADJ) / AV_DENOM
        #HSA_ADJUSTMENT = HSA_MOD_AV
        
    
        
    def avc_14_calc_av(self, tier):
        temp = 0
        get_cost_sharing_parameters() # redundant call, appears in excel macros
        # depending on deductibles and max out of pocket payment, calls different methods to use the 
        # cost sharing parameters and tables given in the excel spreadsheet to calculate AV
        SetContTable_Combined
    SetContTable_Med_and_Rx
    
    If gCB_INT_DEDUCT_IND = 1 Then
    
        CombinedMacro_v2
        CombinedMacro2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 0 Then
    
        SepDeductCombinedMOOP_v2
        SepDeductCombinedMOOP2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 1 Then
        
        SepMacro_v2
        SepMacro2_v2

    End If
       
    CalcAV = gAV
        return temp       
    
    
    
    def avc_15_process_av(self, csr3, csr2, csr1):    
        if self.gCB_CSR_IND = 0:
            if 0.88 <= self.gAV < 0.92:
                self.gACTUAL_MTIER_NUM = 1
                self.gMTIER_NAME = "Platinum"
                return "platinum"
            elif 0.78 <= self.gAV < 0.82:
                self.gACTUAL_MTIER_NUM = 2
                self.gMTIER_NAME = "Gold"
                return "gold"
            elif 0.68 <= self.gAV < 0.72:
                self.gACTUAL_MTIER_NUM = 3
                self.gMTIER_NAME = "Silver"
                return "silver"
            elif 0.58 <= self.gAV < 0.62:
                self.gACTUAL_MTIER_NUM = 4
                self.gMTIER_NAME = "Bronze"
                return "bronze"
            else:
                self.gACTUAL_MTIER_NUM = -1
                self.gMTIER_NAME = ""
                print("Result is outside of +/- 2 percent de minimis variation")
                if 0.93 <= self.gAV < 0.95 and (csr3 or desired_tier == "platinum"):
                    self.gACTUAL_MTIER_NUM = 1
                    self.gMTIER_NAME = "Platinum"
                    metal = "platinum"
                    print("CSR level 94% (100-150% FPL)")
                elif 0.86 <= self.gAV < 0.88 and (csr2 or desired_tier == "gold"):
                    self.gACTUAL_MTIER_NUM = 2
                    self.gMTIER_NAME = "Gold"
                    metal = "gold"
                    print("CSR level 87% (150-200% FPL)")
                elif 0.72 <= self.gAV < 0.74 and (csr1 or desired_tier == "silver"):
                    self.gACTUAL_MTIER_NUM = 3
                    self.gMTIER_NAME = "Silver"
                    metal = "silver"
                    print("CSR Level of 73% (200-250% FPL)")
                elif 0.62 <= self.gAV < 0.65 and (expanded_bronze or desired_tier == "bronze"):
                    self.gACTUAL_MTIER_NUM = 4
                    self.gMTIER_NAME = "Bronze"
                    metal = "bronze"
                    print("Expanded Bronze Standard, 58% to 65%")
                else:
                    raise Exception("Plan does not match up to variation")
                return metal
            return "no_valid_av"
    
    # sets all variables to 0 or null (0 for simplification here)
    def avc_13_reset_vars(self):
        self.gContTableCombined = 0
        self.gContTableMed = 0
        self.gContTableRx = 0
        self.gDEDROW = 0
        self.gDEDROW_MED = 0
        self.gDEDROW_RX = 0
        self.gNoteMsg = 0
        self.gUpToCol = 0
        self.gMaxdCol = 0
        self.gER = 0
        self.gIP = 0
        self.gPC = 0
        self.gPCN = 0
        self.gSP = 0
        self.gPSY = 0
        self.gIMG = 0
        self.gST = 0
        self.gOT = 0
        self.gPREV = 0
        self.gLAB = 0
        self.gSNF = 0
        self.gUNCL = 0
        self.gXRAY = 0
        self.gXRAY_PC = 0
        self.gXRAY_SP = 0
        self.gXRAY_UNC = 0
        self.gOPFAC_PSY = 0
        self.gOPFAC_IMG = 0
        self.gOPFAC_ST = 0
        self.gOPFAC_OT = 0
        self.gOPFAC_LAB = 0
        self.gOPFAC_UNC = 0
        self.gOPPROF_PSY = 0
        self.gOPPROF_IMG = 0
        self.gOPPROF_ST = 0
        self.gOPPROF_OT = 0
        self.gOPPROF_LAB = 0
        self.gOPPROF_SRGRY = 0
        self.gRXGEN = 0
        self.gRXFORM = 0
        self.gRXNONFORM = 0
        self.gRXSPCLTY = 0
        self.gUpToCol_MEDTBL = 0
        self.gMaxdCol_MEDTBL = 0
        self.gER_MEDTBL = 0
        self.gIP_MEDTBL = 0
        self.gPC_MEDTBL = 0
        self.gPCN_MEDTBL = 0
        self.gSP_MEDTBL = 0
        self.gPSY_MEDTBL = 0
        self.gIMG_MEDTBL = 0
        self.gST_MEDTBL = 0
        self.gOT_MEDTBL = 0
        self.gPREV_MEDTBL = 0
        self.gLAB_MEDTBL = 0
        self.gSNF_MEDTBL = 0
        self.gUNCL_MEDTBL = 0
        self.gXRAY_MEDTBL = 0
        self.gXRAY_PC_MEDTBL = 0
        self.gXRAY_SP_MEDTBL = 0
        self.gXRAY_UNC_MEDTBL = 0
        self.gOPFAC_PSY_MEDTBL = 0
        self.gOPFAC_IMG_MEDTBL = 0
        self.gOPFAC_ST_MEDTBL = 0
        self.gOPFAC_OT_MEDTBL = 0
        self.gOPFAC_LAB_MEDTBL = 0
        self.gOPFAC_UNC_MEDTBL = 0
        self.gOPPROF_PSY_MEDTBL = 0
        self.gOPPROF_IMG_MEDTBL = 0
        self.gOPPROF_ST_MEDTBL = 0
        self.gOPPROF_OT_MEDTBL = 0
        self.gOPPROF_LAB_MEDTBL = 0
        self.gOPPROF_SRGRY_MEDTBL = 0
        self.gUpToCol_RXTBL = 0
        self.gMaxdCol_RXTBL = 0
        self.gRXGEN_RXTBL = 0
        self.gRXFORM_RXTBL = 0
        self.gRXNONFORM_RXTBL = 0
        self.gRXSPCLTY_RXTBL = 0
        self.gCombinedLabels_Arr = 0 
        self.gMedicalLabels_Arr = 0
        self.gRxLabels_Arr = 0
        self.gMedical_Arr = 0
        self.gRx_Arr = 0
        self.gMedSvcCopayNR_Arr = 0
        self.gMedSvcCoinsNR_Arr = 0
        self.gRxSvcCopayNR_Arr = 0
        self.gRxSvcCoinsNR_Arr = 0
        self.gSTDED_ARR = 0
        self.gSTCOINS_ARR = 0
        self.gOMIT_ARR = 0
        self.gSTDEDRX_ARR = 0
        self.gSTCOINSRX_ARR = 0
        self.gOMITRX_ARR = 0
        self.gSTATUS_MSG = 0
        self.gERROR_FLAG = 0
        self.gOPFAC_PSY_OVRD = 0
        self.gOPFAC_IMG_OVRD = 0
        self.gOPFAC_ST_OVRD = 0
        self.gOPFAC_OT_OVRD = 0
        self.gOPFAC_LAB_OVRD = 0
        self.gOPPROF_PSY_OVRD = 0
        self.gOPPROF_IMG_OVRD = 0
        self.gOPPROF_ST_OVRD = 0
        self.gOPPROF_OT_OVRD = 0
        self.gOPPROF_LAB_OVRD = 0
        self.gNOTE_XRAY_OVRD = 0
        self.gNOTE_OP_OVRD = 0
        self.gAV = 0
        self.gBENEPAY_COINSRNG = 0
        self.gNESTED_DEDUCT_IND = 0
        self.gPLAN_PAY = 0
        self.gPLAN_PAY_COINSRNG = 0
        self.gTOT_PAY = 0
        self.gTOT_PAY_MED = 0
        self.gTOT_PAY_RX = 0
        self.gBENEPAY_DEDUCT_MED = 0
        self.gBENEPAY_DEDUCT_RX = 0
        self.gBENECOINS_DEDUCT = 0
        self.gDEDUCT_ADJMT = 0
        self.gMOOP_ADJMT = 0
        self.gMOOP_ADJMT_MED = 0
        self.gMOOP_ADJMT_RX = 0
        self.gADJ_DEDUCT = 0
        self.gADJ_DEDUCT_MED = 0
        self.gADJ_DEDUCT_RX = 0
        self.gADJ_MOOP = 0
        self.gADJ_MOOP_MED = 0
        self.gADJ_MOOP_RX = 0
        self.gTROOP = 0
        self.gTROOP_MED = 0
        self.gTROOP_RX = 0
        self.gEFF_COINS = 0
        self.gEFF_COINS_MED = 0
        self.gEFF_COINS_RX = 0
        self.gEFF_COINS_NUM = 0
        self.gEFF_COINS_NUM_MED = 0
        self.gEFF_COINS_NUM_RX = 0
        self.gEFF_COINS_DENOM_MED = 0
        self.gEFF_COINS_DENOM_RX = 0
        self.gAV_DENOM = 0

SyntaxError: invalid syntax (<ipython-input-1-d642a1192572>, line 28)